# CVM. Выбор и проверка критерия для метрик конверсии

## Цели и описание ноутбука
Эксперименты CVM, целевыми метриками которых являются метрики конверсии, могут помочь при решении следующих задач:
- Сравнение конверсии группы с константным значением конверсии
- Сравнение больших конверсий между группами
- Сравнение малых конверсий между группами

Целями данного ноутбука являются:
1. Описание критериев, которые необходимо применять в каждой из вышеописанных задач
2. Описание функций, которые рассчитывают размер выборки и минимальный детектируемый эффект для каждого критерия
3. Описание функций, которые проверяют корректность выбора критерия на синтетических данных
4. Описание функций, которые строят доверительные интервалы для разницы между целевыми метриками
5. Опиание функций, которые используют для проверки статистической значимости при множественном тестировании

In [2]:
# Magic function to autoreload kernel when python module's code is changed
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('white')

from ab_utils.standard_values import z_value, t_value

## 1. Критерии для метрик конверсий

Критерий (или статистический тест) — это правило, по которому на основании выборочных данных принимается решение: принимать или отвергать нулевую гипотезу $H_0$. При этом критерий основывается на случайной величине $T(X_1, X_2, ... , X_n)$, называемой статистикой критерия.

Основные условия, которым должна удовлетворять случайная величина, чтобы считаться критерием:
1. Определена как функция от выборки
2. Известно (или выводимо) распределение при $H_0$
3. Чувствительность к отклонению от $H_0$ - при верной альтернативе $H_1$ распределение $T$ должно изменяться таким образом, чтобы вероятность попасть в критическую область возрастала
4. Монотонность / направленность - для односторонних тестов значение $T$ должно возрастать (или убывать) при переходе от $H_0$ к $H_1$, чтобы можно было задать однозначную «область отклонения» (чем больше отклонение, тем больше мощность теста)
5. Независимость от параметров, не проверяемых гипотезой (инвариантность)
6. Статистика критерия должна быть вычислимой

При решении задач сравнения конверсий необходимо применять следующие критерии:
1. Сравнение конверсии в группе с константным значением:
    - Биномиальный тест (Binomial test)
    - Одновыборочный T-тест (One-sample T-test)
2. Сравнение больших конверсий между группами:
    - Z-тест для пропорций (Z-test)
    - Двухвыборочный T-тест (Welch's T-test)
3. Сравнение малых конверсий между группами:
    - Точный тест Фишера (Fisher's Exact Test)
    - Бутстрап (Bootstrap) (применим ко всем трём задачам)

### 1.1.1. Сравнение конверсии в группе с константой. Биномиальный тест

#### Проверяемые гипотезы
$$
H_0: p = p_0 \\
H_1: p <> p_0
$$
где $p$ - конверсия в экспериментальной группе, $p_0$ - константа, с которой необходимо сравнить конверсию

#### Статистика критерия
Статистикой критерия является количество успехов $k$ в $n$ испытаниях. При верности $H_0$ статистика имеет биномиальное распределение с параметрами $n$ и $p_0$. $k ~ Binomial(n, p_0)$. 

#### Расчёт p-value
$p-value$ в данном случае есть сумма вероятностей всех исходов, вероятность которых меньше либо равна вероятности наблюдаемого $k$.

## 2. Расчёта размера выборки (Sample Size) и минимального детектируемого эффекта (MDE)

...

## 3. Проверка корректности выбора критерия на синтетических данных

...